In [1]:
# import important libraries and download data
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  
# ! git clone https://github.com/Gaurav-Pande/AES_DL.git && mv AES_DL/data .
# ! pip install transformers

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VENKATESH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VENKATESH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VENKATESH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_accuracy_curve(history):
  import matplotlib.pyplot as plt
  plt.plot(history.history['loss'])
  plt.plot(history.history['mae'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

def plot_acrchitecture(filename, model):
  from keras.utils import plot_model
  plot_model(model, to_file=str(filename) + '.png')

In [3]:
# method to split data into sets
def split_in_sets(data):
    essay_sets = []
    min_scores = []
    max_scores = []
    for s in range(1,9):
        essay_set = data[data["essay_set"] == s]
        essay_set.dropna(axis=1, inplace=True)
        n, d = essay_set.shape
        set_scores = essay_set["domain1_score"]
        print ("Set", s, ": Essays = ", n , "\t Attributes = ", d)
        min_scores.append(set_scores.min())
        max_scores.append(set_scores.max())
        essay_sets.append(essay_set)
    return (essay_sets, min_scores, max_scores)

In [7]:
dataset_path = "D:/NEU/DL/Project/Essay grading/English-Language-Learning-Prediction-with-AI-and-Machine-Learning/ASAP Dataset/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
# data.head()

min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set2.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set5.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set8.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
data.head()

Set 1 : Essays =  1783 	 Attributes =  6
Set 2 : Essays =  1800 	 Attributes =  9
Set 3 : Essays =  1726 	 Attributes =  6
Set 4 : Essays =  1770 	 Attributes =  6
Set 5 : Essays =  1805 	 Attributes =  6
Set 6 : Essays =  1800 	 Attributes =  6
Set 7 : Essays =  1569 	 Attributes =  14
Set 8 : Essays =  723 	 Attributes =  18


C:\Users\VENKATESH\AppData\Local\Temp\ipykernel_50396\3722479573.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)
C:\Users\VENKATESH\AppData\Local\Temp\ipykernel_50396\3722479573.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)
C:\Users\VENKATESH\AppData\Local\Temp\ipykernel_50396\3722479573.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [8]:
from keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K

def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.5, recurrent_dropout=0.4, input_size=768, activation='relu', bidirectional = False):
    """Define the model."""
    model = Sequential()
    if bidirectional:
        model.add(Bidirectional(LSTM(Hidden_dim1,return_sequences=return_sequences , dropout=0.4, recurrent_dropout=recurrent_dropout), input_shape=[1, input_size]))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout)))
    else:
        model.add(LSTM(Hidden_dim1, dropout=0.4, recurrent_dropout=recurrent_dropout, input_shape=[1, input_size], return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation=activation))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

def get_model_CNN(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.5, recurrent_dropout=0.4, input_size=768,output_dims=10380, activation='relu', bidirectional = False):
    """Define the model."""
    inputs = Input(shape=(768,1))
    x = Conv1D(64, 3, strides=1, padding='same', activation='relu')(inputs)
    #Cuts the size of the output in half, maxing over every 2 inputs
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(128, 3, strides=1, padding='same', activation='relu')(x)
    x = GlobalMaxPooling1D()(x) 
    outputs = Dense(output_dims, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs, name='CNN')
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])
    model.summary()
    return model

In [ ]:

## Sets experiment BERT
import time
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []

for s in sets:
  print("\n--------SET {}--------\n".format(set_count))
  X = s
  y = s['domain1_score']
  cv = KFold(n_splits=5, shuffle=True)
  cv_data = cv.split(X)
  results = []
  prediction_list = []
  fold_count =1
  cuda = torch.device('cuda')
  # For DistilBERT:
  model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
  ## Want BERT instead of distilBERT? Uncomment the following line:
  ##model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
  # Load pretrained model/tokenizer
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
  model = model_class.from_pretrained(pretrained_weights)
  with torch.cuda.device(cuda):
    for traincv, testcv in cv_data:
      torch.cuda.empty_cache()
      print("\n--------Fold {}--------\n".format(fold_count))
      # get the train and test from the dataset.
      X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
      train_essays = X_train['essay']
      #print("y_train",y_train)
      test_essays = X_test['essay']
      # model = model.cuda()
      #y_train = torch.tensor(y_train,dtype=torch.long)
      sentences = []
      tokenize_sentences = []
      train_bert_embeddings = []
      #bert_embedding = BertEmbedding()
      # for essay in train_essays:
      #   # get all the sentences from the essay
      #   sentences += essay_to_sentences(essay, remove_stopwords = True)
      # sentences = pd.Series(sentences)
      # print(train_essays)
      tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
      tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))


      ## train
      max_len = 0
      for i in tokenized_train.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

      attention_mask_train = np.where(padded_train != 0, 1, 0)



      train_input_ids = torch.tensor(padded_train)
      train_attention_mask = torch.tensor(attention_mask_train)
      with torch.no_grad():
        last_hidden_states_train = model(train_input_ids, attention_mask=train_attention_mask)


      train_features = last_hidden_states_train[0][:,0,:].numpy()
      

      ## test
      max_len = 0
      for i in tokenized_test.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
      attention_mask_test = np.where(padded_test != 0, 1, 0)
      test_input_ids = torch.tensor(padded_test)  
      test_attention_mask = torch.tensor(attention_mask_test)

      with torch.no_grad():
        last_hidden_states_test = model(test_input_ids, attention_mask=test_attention_mask)

      test_features = last_hidden_states_test[0][:,0,:].numpy()
      



      train_x,train_y = train_features.shape
      test_x,test_y = test_features.shape

      trainDataVectors = np.reshape(train_features,(train_x,1,train_y))
      testDataVectors = np.reshape(test_features,(test_x,1,test_y))

      lstm_model = get_model(bidirectional=False)
      lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=70)
      y_pred = lstm_model.predict(testDataVectors)

      y_pred = np.around(y_pred)
      #y_pred.dropna()
      np.nan_to_num(y_pred)
      # evaluate the model
      result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
      print("Kappa Score: {}".format(result))
      results.append(result)
      fold_count +=1
      import tensorflow as tf
      tf.keras.backend.clear_session()



  all_sets_score.append(results)
  print("Average kappa score value is : {}".format(np.mean(np.asarray(results))))
  set_count+=1
    # print(features.shape)

In [ ]:

## Sets experiment BERT
import time
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []

for s in sets:
  print("\n--------SET {}--------\n".format(set_count))
  X = s
  y = s['domain1_score']
  cv = KFold(n_splits=5, shuffle=True)
  cv_data = cv.split(X)
  results = []
  prediction_list = []
  fold_count =1
  cuda = torch.device('cuda')
  # For DistilBERT:
  model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
  ## Want BERT instead of distilBERT? Uncomment the following line:
  ##model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
  # Load pretrained model/tokenizer
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
  model = model_class.from_pretrained(pretrained_weights)
  with torch.cuda.device(cuda):
    for traincv, testcv in cv_data:
      torch.cuda.empty_cache()
      print("\n--------Fold {}--------\n".format(fold_count))
      # get the train and test from the dataset.
      X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
      train_essays = X_train['essay']
      #print("y_train",y_train)
      test_essays = X_test['essay']
      # model = model.cuda()
      #y_train = torch.tensor(y_train,dtype=torch.long)
      sentences = []
      tokenize_sentences = []
      train_bert_embeddings = []
      #bert_embedding = BertEmbedding()
      # for essay in train_essays:
      #   # get all the sentences from the essay
      #   sentences += essay_to_sentences(essay, remove_stopwords = True)
      # sentences = pd.Series(sentences)
      # print(train_essays)
      tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
      tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))


      ## train
      max_len = 0
      for i in tokenized_train.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

      attention_mask_train = np.where(padded_train != 0, 1, 0)



      train_input_ids = torch.tensor(padded_train)
      train_attention_mask = torch.tensor(attention_mask_train)
      with torch.no_grad():
        last_hidden_states_train = model(train_input_ids, attention_mask=train_attention_mask)


      train_features = last_hidden_states_train[0][:,0,:].numpy()
      

      ## test
      max_len = 0
      for i in tokenized_test.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
      attention_mask_test = np.where(padded_test != 0, 1, 0)
      test_input_ids = torch.tensor(padded_test)  
      test_attention_mask = torch.tensor(attention_mask_test)

      with torch.no_grad():
        last_hidden_states_test = model(test_input_ids, attention_mask=test_attention_mask)

      test_features = last_hidden_states_test[0][:,0,:].numpy()
      



      train_x,train_y = train_features.shape
      test_x,test_y = test_features.shape

      trainDataVectors = np.reshape(train_features,(train_x,1,train_y))
      testDataVectors = np.reshape(test_features,(test_x,1,test_y))

      lstm_model = get_model(bidirectional=False)
      lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=70)
      y_pred = lstm_model.predict(testDataVectors)

      y_pred = np.around(y_pred)
      #y_pred.dropna()
      np.nan_to_num(y_pred)
      # evaluate the model
      result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
      print("Kappa Score: {}".format(result))
      results.append(result)
      fold_count +=1
      import tensorflow as tf
      tf.keras.backend.clear_session()



  all_sets_score.append(results)
  print("Average kappa score value is : {}".format(np.mean(np.asarray(results))))
  set_count+=1
    # print(features.shape)

In [ ]:

## Sets experiment BERT
import time
import torch
import transformers as ppb
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []
all_sets_avg=[]
for s in sets:
  print("\n--------SET {}--------\n".format(set_count))
  X = s
  y = s['domain1_score']
  cv = KFold(n_splits=5, shuffle=True)
  cv_data = cv.split(X)
  results = []
  prediction_list = []
  fold_count =1
  cuda = torch.device('cuda')
  # For DistilBERT:
  model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
  ## Want BERT instead of distilBERT? Uncomment the following line:
  ##model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
  # Load pretrained model/tokenizer
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
  model = model_class.from_pretrained(pretrained_weights)
  with torch.cuda.device(cuda):
    for traincv, testcv in cv_data:
      torch.cuda.empty_cache()
      print("\n--------Fold {}--------\n".format(fold_count))
      # get the train and test from the dataset.
      X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
      train_essays = X_train['preprocessed_text']
      #print("y_train",y_train)
      test_essays = X_test['preprocessed_text']
      # model = model.cuda()
      #y_train = torch.tensor(y_train,dtype=torch.long)
      sentences = []
      tokenize_sentences = []
      train_bert_embeddings = []
      #bert_embedding = BertEmbedding()
      # for essay in train_essays:
      #   # get all the sentences from the essay
      #   sentences += essay_to_sentences(essay, remove_stopwords = True)
      # sentences = pd.Series(sentences)
      # print(train_essays)
      tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
      tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))


      ## train
      max_len = 0
      for i in tokenized_train.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

      attention_mask_train = np.where(padded_train != 0, 1, 0)



      train_input_ids = torch.tensor(padded_train)
      train_attention_mask = torch.tensor(attention_mask_train)
      with torch.no_grad():
        last_hidden_states_train = model(train_input_ids, attention_mask=train_attention_mask)


      train_features = last_hidden_states_train[0][:,0,:].numpy()
      

      ## test
      max_len = 0
      for i in tokenized_test.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
      attention_mask_test = np.where(padded_test != 0, 1, 0)
      test_input_ids = torch.tensor(padded_test)  
      test_attention_mask = torch.tensor(attention_mask_test)

      with torch.no_grad():
        last_hidden_states_test = model(test_input_ids, attention_mask=test_attention_mask)

      test_features = last_hidden_states_test[0][:,0,:].numpy()
      



      train_x,train_y = train_features.shape
      test_x,test_y = test_features.shape

      trainDataVectors = np.reshape(train_features,(train_x,1,train_y))
      testDataVectors = np.reshape(test_features,(test_x,1,test_y))

      lstm_model = get_model(bidirectional=False)
      lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=70)
      y_pred = lstm_model.predict(testDataVectors)

      y_pred = np.around(y_pred)
      #y_pred.dropna()
      np.nan_to_num(y_pred)
      # evaluate the model
      result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
      print("Kappa Score: {}".format(result))
      results.append(result)
      fold_count +=1
      
      tf.keras.backend.clear_session()



  all_sets_score.append(results)
  avg=np.mean(np.asarray(results))
  print("Average kappa score value is : {}".format(avg))
  all_sets_avg.append(avg)
  set_count+=1
    # print(features.shape)

In [ ]:
#XLNET

## Sets experiment BERT
import time
import torch
import transformers as ppb
import tensorflow as tf
import warnings

warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []
all_sets_avg=[]

for s in sets:
  print("\n--------SET {}--------\n".format(set_count))
  X = s
  y = s['domain1_score']
  cv = KFold(n_splits=5, shuffle=True)
  cv_data = cv.split(X)
  results = []
  prediction_list = []
  fold_count =1
  cuda = torch.device('cuda')

  # For DistilBERT:
  model_class, tokenizer_class, pretrained_weights = (ppb.XLNetModel, ppb.XLNetTokenizer, 'xlnet-base-cased')
  
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
  model = model_class.from_pretrained(pretrained_weights)

  with torch.cuda.device(cuda):
    for traincv, testcv in cv_data:
      torch.cuda.empty_cache()
      print("\n--------Fold {}--------\n".format(fold_count))

      
      X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
      train_essays = X_train['preprocessed_text']
      
      test_essays = X_test['preprocessed_text']
      
      sentences = []
      tokenize_sentences = []
      train_bert_embeddings = []
      tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
      tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))


      ## train
      max_len = 0
      for i in tokenized_train.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

      attention_mask_train = np.where(padded_train != 0, 1, 0)



      train_input_ids = torch.tensor(padded_train)
      train_attention_mask = torch.tensor(attention_mask_train)
      with torch.no_grad():
        # last_hidden_states_train = model(train_input_ids, attention_mask=train_attention_mask)
        outputs= model(train_input_ids, attention_mask=train_attention_mask)
        last_hidden_states_train = outputs.last_hidden_state




      train_features = last_hidden_states_train[0][:,0,:].numpy()
      

      ## test
      max_len = 0
      for i in tokenized_test.values:
        if len(i) > max_len:
          max_len = len(i)
      padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
      attention_mask_test = np.where(padded_test != 0, 1, 0)
      test_input_ids = torch.tensor(padded_test)  
      test_attention_mask = torch.tensor(attention_mask_test)

      with torch.no_grad():
        # last_hidden_states_test = model(test_input_ids, attention_mask=test_attention_mask)
        outputs= model(test_input_ids, attention_mask=test_attention_mask)
        last_hidden_states_test = outputs.last_hidden_state

      test_features = last_hidden_states_test[0][:,0,:].numpy()
      

      train_x,train_y = train_features.shape
      test_x,test_y = test_features.shape

      trainDataVectors = np.reshape(train_features,(train_x,1,train_y))
      testDataVectors = np.reshape(test_features,(test_x,1,test_y))

      lstm_model = get_model(bidirectional=False)
      lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=10)
      y_pred = lstm_model.predict(testDataVectors)

      y_pred = np.around(y_pred)
      #y_pred.dropna()
      np.nan_to_num(y_pred)
      # evaluate the model
      result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
      print("Kappa Score: {}".format(result))
      results.append(result)
      fold_count +=1
      
      tf.keras.backend.clear_session()



  all_sets_score.append(results)
  avg=np.mean(np.asarray(results))
  print("Average kappa score value is : {}".format(avg))
  all_sets_avg.append(avg)
  set_count+=1
    # print(features.shape)